In [1]:
!pip install --upgrade tokenizers sentencepiece transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 115.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.4 MB/s eta 0:00:00


In [7]:
from google.colab import drive
from IPython.display import display
from IPython.html import widgets
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torch import optim
from torch.nn import functional as F
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm_notebook

/usr/local/lib/python3.10/dist-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [2]:
translated_model=[]
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM
model_repo = 'google/mt5-small'
model_path = '/content/drive/MyDrive/MT5/v2MT5_AR_EN.pt'
max_seq_len = 20

In [3]:
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'ar': '<ar>'
}

In [4]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer = AutoTokenizer.from_pretrained(model_repo)
tokenizer.add_special_tokens(special_tokens_dict)
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Embedding(250102, 512)

In [8]:
model.cuda()
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [9]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

all= wn.all_synsets()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

In [ ]:
arabic=[]
count = 999
for synset in all :
  data = synset.definition()

  input_ids = encode_input_str(
    text = data,
    target_lang = 'ar',
    tokenizer = tokenizer,
    seq_len = 64,
    lang_token_map = LANG_TOKEN_MAPPING)
  
  input_ids = input_ids.unsqueeze(0).cuda()

  output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=1)
  str_out=''
  for token_set in output_tokens:
    str_out += tokenizer.decode(token_set, skip_special_tokens=True)  
  arabic.append(str_out)
  print(count)
  count-=1

In [13]:
print(arabic[:100])

['(يتبع ذلك عادة أنه لا يجوز أن يكون هناك', 'وهو يواجه بعيداً عن نطاق عضو أو كائن أو ', 'أقرب إلى أو مواجهة على نطاق أنظمة أو هياكل', 'مواجهة أو على الجانب على الجانب الذي يتوجه إلى', 'مواجهة أو على جانب القاعدة أو على جانبها', 'وعلى وجه الخصوص، ينبغي على الجسد أن يبعد', 'خاصة بالعضلات، التي تجمع في منتصف الجسم أو نحو جز', 'أن يولد أو يولد أو يَبْدأُ.', 'يأتي إلى الوجود', 'يَجِبُ أَنْ يَمْكِن', 'إعطاء الولادة', 'ويجب أن يكون على نطاق واسع على نطاق واسع', 'كوني على نقطة الموت تتنفسي أخيراً', 'occurring عند وقت الموت', '(يمكن الاطلاع على نسخة من المادة) التي يمكن', 'مع أجزاء مفتوحة', 'يمتد إلى نصف طوله الأصلي', '(اللغة البريطانية) مفصلة أو مفصلة أو مفصلة', '(يمكن الاطلاع على النصات) على نطاق واسع ', 'كامل', 'مثالية أو كاملة أو كاملة أو كاملة', 'وإزعاج عناصر التهديد أو التهديد؛ تماماً؛ ', 'بدون شك أو بدون إهتمام أو بدون إهتمام أو احتياطي', 'تماماً وجميعاً يؤدي إلى تعبئة جميع', '(اللغة غير الرسمية) (اللغة غير الرسمية)', 'يقدر على مقارنة؛ وليس على نطاق واسع؛', 'وجود علاقة أو علاقة على علاقة

In [14]:
with open('/content/drive/MyDrive/Final folder/final files/englishModel.txt','w',encoding='utf-8') as f:
  for idx, e in enumerate(arabic) :
    if (idx <=999):
      data = e

      input_ids = encode_input_str(
        text = data,
        target_lang = 'en',
        tokenizer = tokenizer,
        seq_len = 20,
        lang_token_map = LANG_TOKEN_MAPPING)
      
      input_ids = input_ids.unsqueeze(0).cuda()

      output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=1)
      str_out=''
      for token_set in output_tokens:
        str_out += tokenizer.decode(token_set, skip_special_tokens=True)  
      f.write(str_out)
      f.write('\n')
      print(idx)
    else :
      break

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27